In [243]:
import pandas as pd

In [244]:
import numpy as np

Logic :
1. Jika settlement amount not null dan settlement time null maka ini adalah realtime
2. Jika settlement amount not null dan settlement time not null maka ini adalah non realtime

In [245]:
gds = pd.read_csv('Exclude Nobu 26 May 24.csv')

/var/folders/sg/v7nz4brs3sq_pn793b_nw89c0000gp/T/ipykernel_21311/2887963253.py:1: DtypeWarning: Columns (14,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  gds = pd.read_csv('Exclude Nobu 26 May 24.csv')


In [246]:
gds2 = pd.read_csv('Nobu 26 May 24.csv')

In [247]:
# gds3 = pd.read_csv('Exclude Nobu 23 May 24.csv')

In [248]:
# gds4 = pd.read_csv('Exclude Nobu 24 May 24.csv')

In [249]:
# gds5 = pd.read_csv('Nobu 21 May 24.csv')

In [250]:
# gds6 = pd.read_csv('Nobu 22 May 24.csv')

In [251]:
# gds7 = pd.read_csv('Nobu 23 May 24.csv')

In [252]:
# gds8 = pd.read_csv('Nobu 24 May 24.csv')

In [253]:
df1 = pd.concat([gds,gds2]).reset_index(drop=True)

In [254]:
df1['settlement_amount'].sum()

55825441433

In [255]:
#len(gds) + len(gds2) + len(gds3) + len(gds4) + len(gds5)

In [256]:
df1.dtypes

created_datetime                          object
last_updated_datetime                     object
settlement_time                           object
settlement_amount                          int64
username                                  object
product_type                              object
charge_vendor_code                        object
charge_status                             object
admin_fee                                  int64
base_admin_fee                             int64
admin_fee_invoice                          int64
deduction_cost                           float64
deduction_cost_gross                     float64
transaction_amount                         int64
merchant_id                               object
id                                        object
qr_id                                     object
qris_vendor_creation_reference_number     object
Transaction Channel Type                  object
Transaction SN                            object
dtype: object

In [257]:
df1['last_updated_datetime'] = pd.to_datetime(df1['last_updated_datetime'])

/var/folders/sg/v7nz4brs3sq_pn793b_nw89c0000gp/T/ipykernel_21311/1282741160.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df1['last_updated_datetime'] = pd.to_datetime(df1['last_updated_datetime'])


In [258]:
df1['settlement_time'] = pd.to_datetime(df1['settlement_time'])

/var/folders/sg/v7nz4brs3sq_pn793b_nw89c0000gp/T/ipykernel_21311/997221283.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df1['settlement_time'] = pd.to_datetime(df1['settlement_time'])


In [259]:
df1['last_updated_datetime'] = pd.to_datetime(df1['last_updated_datetime']).dt.date

In [260]:
df1['realtime?'] = np.nan

Cek Null Data

In [261]:
isnull = df1[(df1['settlement_amount'].isna() & df1['settlement_time'].isna()) | df1['settlement_amount'].isna()]

Cek username yang null

In [262]:
isnull['username'].unique()

array([], dtype=object)

Jika kolom settlement amount & settlement time null data, maka raise ke BI dulu (Buffer T+3 dulu), baru lanjut

In [263]:
len(df1)

714988

Cek realtime settlement username

In [264]:
realtimeindex = df1[df1['settlement_time'].isna() & df1['settlement_amount'].notna()].index

In [265]:
realtimeindex

Index([     0,      1,      2,      4,      5,      6,      7,      8,     11,
           12,
       ...
       573549, 573550, 573552, 573553, 573554, 573555, 573556, 573557, 573558,
       573559],
      dtype='int64', length=295515)

In [266]:
df1.loc[realtimeindex, 'settlement_time'] = df1['last_updated_datetime']

In [267]:
df1.loc[realtimeindex, 'settlement_time'] = pd.to_datetime(df1['settlement_time'].dt.date)

In [268]:
df1.loc[realtimeindex,'realtime?'] = "Realtime"

In [269]:
nonrealtimeindex = df1[df1['realtime?'].isna()].index

In [270]:
df1.loc[nonrealtimeindex, 'realtime?'] = 'Non Realtime'

In [271]:
df1

,created_datetime,last_updated_datetime,settlement_time,settlement_amount,username,product_type,charge_vendor_code,charge_status,admin_fee,base_admin_fee,...,deduction_cost,deduction_cost_gross,transaction_amount,merchant_id,id,qr_id,qris_vendor_creation_reference_number,Transaction Channel Type,Transaction SN,realtime?
0,"May 26, 2024, 11:59:43 PM",2024-05-26,2024-05-26 00:00:00,79440,nbatech,Payment Routing Acceptance,DANAMON,SUCCESS,560,505,...,340.000,560.000,80000,1039542942,2405LLFT4AAX,282068571524829,80810296,OFF_US,000435738496,Realtime
1,"May 26, 2024, 11:59:41 PM",2024-05-26,2024-05-26 00:00:00,9930,nbatech,Payment Routing Acceptance,DANAMON,SUCCESS,70,63,...,42.500,70.000,10000,1039542942,2405O4ISXGWY,49434976689080,80810291,OFF_US,000435738441,Realtime
2,"May 26, 2024, 11:59:39 PM",2024-05-26,2024-05-26 00:00:00,137943,bhavanatech,Payment Routing Acceptance,DANAMON,SUCCESS,1057,952,...,590.750,973.000,139000,1039652562,2405AZNWKYBK,561629240842768,80810289,OFF_US,405663768813,Realtime
3,"May 26, 2024, 11:59:36 PM",2024-05-26,2024-05-28 15:00:00,19998,lightyear,Payment Routing Acceptance,DANAMON,SUCCESS,198,178,...,85.833,141.372,20196,1039296223,2405WSJ3E1JD,66294657793645,80810282,OFF_US,12bob3l31829,Non Realtime
4,"May 26, 2024, 11:59:31 PM",2024-05-26,2024-05-26 00:00:00,44685,nbatech,Payment Routing Acceptance,DANAMON,SUCCESS,315,284,...,191.250,315.000,45000,1039542942,240519LU2GE2,776425543343194,80810278,OFF_US,12bob3a35529,Realtime
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714983,"May 25, 2024, 11:58:13 PM",2024-05-26,2024-05-27 12:00:00,49400,kksabila07,Payment Routing Acceptance,NOBU,SUCCESS,600,541,...,255.500,350.000,50000,936005030000080783,2405RV486QPB,0I3YZNVX92JP2VVP2BSF,NaN,OFF_US,012159174867,Non Realtime
714984,"May 25, 2024, 11:57:57 PM",2024-05-26,2024-05-27 12:00:00,49610,darthvader,Payment Checkout,NOBU,SUCCESS,390,351,...,255.500,350.000,50000,936005030000080783,24059O7X7VEO,09V3KAJPU1VRB5TQBMQ7,NaN,OFF_US,011871747240,Non Realtime
714985,"May 25, 2024, 11:55:51 PM",2024-05-26,2024-05-27 12:00:00,14883,darthvader,Payment Checkout,NOBU,SUCCESS,117,105,...,76.650,105.000,15000,936005030000080783,24053Q3SIXFY,01T7D7WQBRSXLROYIZSM,NaN,OFF_US,011901159731,Non Realtime
714986,"May 25, 2024, 11:51:37 PM",2024-05-26,2024-05-27 12:00:00,24805,darthvader,Payment Checkout,NOBU,SUCCESS,195,176,...,127.750,175.000,25000,936005030000080783,24053L70IOML,0SS7PO52V1GB68K3L9HQ,NaN,OFF_US,011896223298,Non Realtime


In [272]:
grouping = df1.groupby(['charge_vendor_code','username', 'realtime?', 'last_updated_datetime', 'settlement_time']).agg({'settlement_amount' : 'sum', 'qr_id' : 'count'})

In [273]:
grouping = grouping.reset_index()

In [274]:
grouping.rename(columns={'charge_vendor_code' : 'Vendor Name', 'last_updated_datetime' : 'Transaction Date', 'qr_id' : '#Trx'}, inplace=True)

In [275]:
grouping = grouping.sort_values(by=['realtime?'])

In [276]:
nobu = grouping[grouping['Vendor Name'] == "NOBU"]

In [277]:
danamon = grouping[grouping['Vendor Name'] == "DANAMON"]

In [278]:
bagi = grouping[grouping['Vendor Name'] == "BAGI"]

In [279]:
dana = grouping[grouping['Vendor Name'] == "DANA"]

In [280]:
gv = grouping[grouping['Vendor Name'] == "GV"]

In [281]:
danamon

,Vendor Name,username,realtime?,Transaction Date,settlement_time,settlement_amount,#Trx
33,DANAMON,makmurapp,Non Realtime,2024-05-26,2024-05-27 12:00:00,5810122,48
35,DANAMON,ottosolutions,Non Realtime,2024-05-26,2024-05-27 12:00:00,24843703,247
37,DANAMON,skyzeevoucher,Non Realtime,2024-05-26,2024-05-27 12:00:00,2655000,10
38,DANAMON,solusitekindo,Non Realtime,2024-05-26,2024-05-27 12:00:00,7504726,94
39,DANAMON,spnpay,Non Realtime,2024-05-26,2024-05-27 12:00:00,36878034,242
40,DANAMON,tangankanan,Non Realtime,2024-05-26,2024-05-27 12:00:00,28516749,247
41,DANAMON,tarabyte,Non Realtime,2024-05-26,2024-05-27 12:00:00,534196918,6374
42,DANAMON,taracore,Non Realtime,2024-05-26,2024-05-27 12:00:00,616683034,6359
43,DANAMON,taratech,Non Realtime,2024-05-26,2024-05-27 12:00:00,457514488,6718
44,DANAMON,teknomart,Non Realtime,2024-05-26,2024-05-27 12:00:00,49601,1


In [282]:
nobu['settlement_amount'].sum() + danamon['settlement_amount'].sum() + bagi['settlement_amount'].sum() + dana['settlement_amount'].sum() + gv['settlement_amount'].sum()

55825441433

Start E_wallet

In [46]:
gdse1 = pd.read_csv('All EWallet 26 May 24.csv')

In [47]:
# gdse2 = pd.read_csv('All Ewallet 22 May 24.csv')

In [48]:
# gdse3 = pd.read_csv('All Ewallet 23 May 24.csv')

In [49]:
# gdse4 = pd.read_csv('All Ewallet 24 May 24.csv')

In [50]:
# df2 = pd.concat([gdse1,gdse2,gdse3,gdse4]).reset_index(drop=True)

In [51]:
df2 = gdse1

In [52]:
df2

,created_datetime,last_updated_datetime,settlement_time,source_of_fund,service,vendor_code,transaction_id,tx_ref_number,tx_serial_number,amount,username,transaction_status,vendor_service,admin_fee,admin_fee_invoice,deduction_cost,settlement_amount
0,"May 26, 2024, 11:59:50 PM","May 26, 2024, 11:59:57 PM","May 27, 2024, 12:00:00 PM",EWALLET,E-Wallet Aggregator,DANA,dec33f48-aa3d-48f5-afb9-5479183deaaa,2405QN08Q3JD,20240526111212800110166121202413820,100000,abhitech,SUCCESS,H2H,1500,0,1300.00,98500
1,"May 26, 2024, 11:59:49 PM","May 26, 2024, 11:59:58 PM",NaN,EWALLET,E-Wallet Aggregator,DANA,d1027eeb-82f1-48c8-b846-cf965d6d2c84,2405E2YAPOJK,20240526111212800110166122502409037,10000,nbatech,SUCCESS,H2H,180,0,130.00,9820
2,"May 26, 2024, 11:59:48 PM","May 26, 2024, 11:59:59 PM","May 28, 2024, 3:00:00 PM",EWALLET,E-Wallet Aggregator,DANA,288ddbc2-a656-4113-ac58-c7d0f52677a4,24053WNM0YGB,20240526111212800110166588202037021,50000,anjanajayateknologi,SUCCESS,H2H,755,0,650.00,49245
3,"May 26, 2024, 11:59:47 PM","May 26, 2024, 11:59:53 PM","May 27, 2024, 12:00:00 PM",EWALLET,E-Wallet Aggregator,DANA,0e5da796-6510-404a-ab39-2689fae0af57,2405VN0YZ99T,20240526111212800110166620602034603,20000,anjanajayateknologi,SUCCESS,H2H,302,0,260.00,19698
4,"May 26, 2024, 11:59:46 PM","May 26, 2024, 11:59:57 PM","May 27, 2024, 12:00:00 PM",EWALLET,E-Wallet Aggregator,DANA,d09207d0-e399-49aa-aad8-6527c70a1ed5,2405DDILF9HJ,20240526111212800110166778801994173,20000,anjanajayateknologi,SUCCESS,H2H,302,0,260.00,19698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323926,"May 25, 2024, 11:55:56 PM","May 26, 2024, 12:37:49 AM",NaN,EWALLET,E-Wallet Aggregator,DANA,3203745a-0236-49b1-a219-05589177438a,240599SVDPPC,20240525111212800110166010702309815,58000,gamingku,SUCCESS,H2H,1044,0,754.00,56956
323927,"May 25, 2024, 11:49:45 PM","May 26, 2024, 12:00:02 AM","May 27, 2024, 12:00:00 PM",EWALLET,E-Wallet Aggregator,DANA,86720638-6240-4b19-ab0d-b184b16f8d76,2405P5T092JG,20240525111212800110166427202403969,20000,abhitech,SUCCESS,H2H,300,0,260.00,19700
323928,"May 25, 2024, 11:49:19 PM","May 26, 2024, 12:05:54 AM","May 27, 2024, 12:00:00 PM",EWALLET,E-Wallet Aggregator,DANA,ae9ef2d3-7780-47c9-b290-cfd25cc58664,2405AE0N40W7,20240525111212800110166795601952852,103130,abhitech,SUCCESS,H2H,1547,0,1340.69,101583
323929,"May 25, 2024, 11:44:19 PM","May 26, 2024, 12:00:22 AM","May 27, 2024, 12:00:00 PM",EWALLET,E-Wallet Aggregator,DANA,d4bafba9-6d28-4423-8bf9-708a3fddc2b7,24052XJHASXR,20240525111212800110166974501871645,50000,abhitech,SUCCESS,H2H,750,0,650.00,49250


In [53]:
df2['settlement_amount'].sum()

15915213738

In [54]:
df2['last_updated_datetime'] = pd.to_datetime(df2['last_updated_datetime'])

/var/folders/sg/v7nz4brs3sq_pn793b_nw89c0000gp/T/ipykernel_21311/373268901.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df2['last_updated_datetime'] = pd.to_datetime(df2['last_updated_datetime'])


In [55]:
df2['settlement_time'] = pd.to_datetime(df2['settlement_time'])

In [56]:
df2['last_updated_datetime'] = pd.to_datetime(df2['last_updated_datetime']).dt.date

In [57]:
df2['realtime?'] = np.nan

In [58]:
df2[df2['settlement_amount'].isna()]

,created_datetime,last_updated_datetime,settlement_time,source_of_fund,service,vendor_code,transaction_id,tx_ref_number,tx_serial_number,amount,username,transaction_status,vendor_service,admin_fee,admin_fee_invoice,deduction_cost,settlement_amount,realtime?


In [59]:
isnull2 = df2[(df2['settlement_amount'].isna() & df2['settlement_time'].isna()) | df2['settlement_amount'].isna()]

In [60]:
isnull2['username'].unique()

array([], dtype=object)

In [61]:
realtimeindex = df2[df2['settlement_time'].isna() & df2['settlement_amount'].notna()].index

In [62]:
df2[df2['settlement_time'].isna()]

,created_datetime,last_updated_datetime,settlement_time,source_of_fund,service,vendor_code,transaction_id,tx_ref_number,tx_serial_number,amount,username,transaction_status,vendor_service,admin_fee,admin_fee_invoice,deduction_cost,settlement_amount,realtime?
1,"May 26, 2024, 11:59:49 PM",2024-05-26,NaT,EWALLET,E-Wallet Aggregator,DANA,d1027eeb-82f1-48c8-b846-cf965d6d2c84,2405E2YAPOJK,20240526111212800110166122502409037,10000,nbatech,SUCCESS,H2H,180,0,130.0,9820,NaN
6,"May 26, 2024, 11:59:45 PM",2024-05-26,NaT,EWALLET,E-Wallet Aggregator,DANA,43040f32-6dfc-4973-ba6d-b252863817b1,2405Y6FZHYJB,20240526111212800110166730301999367,40000,gamingku,SUCCESS,H2H,720,0,520.0,39280,NaN
8,"May 26, 2024, 11:59:45 PM",2024-05-26,NaT,EWALLET,E-Wallet Aggregator,DANA,26183cac-09f8-4dd5-bf92-9e3137b3db6a,2405ZUAUGQK1,20240526111212800110166220402371970,56300,gamingku,SUCCESS,H2H,1014,0,731.9,55286,NaN
10,"May 26, 2024, 11:59:44 PM",2024-05-26,NaT,EWALLET,E-Wallet Aggregator,DANA,27086d75-99b5-43cf-96d6-9812caaedaa7,2405QVJLK3XN,20240526111212800110166883301923981,10000,nbatech,SUCCESS,H2H,180,0,130.0,9820,NaN
17,"May 26, 2024, 11:59:39 PM",2024-05-26,NaT,EWALLET,E-Wallet Aggregator,DANA,85a0f4b2-dca1-4543-a314-942b8961f122,2405N1XBAD7V,20240526111212800110166907202071471,10000,nbatech,SUCCESS,H2H,180,0,130.0,9820,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323923,"May 25, 2024, 11:57:53 PM",2024-05-26,NaT,EWALLET,E-Wallet Aggregator,DANA,4189b2ba-bb3b-425e-bb70-27cdfacb5875,24053LKAI8K3,20240525111212800110166915101806948,20000,nbatech,SUCCESS,H2H,360,0,260.0,19640,NaN
323924,"May 25, 2024, 11:57:15 PM",2024-05-26,NaT,EWALLET,E-Wallet Aggregator,DANA,1b3bea5b-14dc-4f3e-81b2-9bdf4cb0b5fd,24051RU76ILC,20240525111212800110166803402054228,30000,gamingku,SUCCESS,H2H,540,0,390.0,29460,NaN
323925,"May 25, 2024, 11:56:08 PM",2024-05-26,NaT,EWALLET,E-Wallet Aggregator,DANA,1b927481-8636-4b8e-beac-4ac887c9b655,2405QE10V5RQ,20240525111212800110166961901967128,10000,nbatech,SUCCESS,H2H,180,0,130.0,9820,NaN
323926,"May 25, 2024, 11:55:56 PM",2024-05-26,NaT,EWALLET,E-Wallet Aggregator,DANA,3203745a-0236-49b1-a219-05589177438a,240599SVDPPC,20240525111212800110166010702309815,58000,gamingku,SUCCESS,H2H,1044,0,754.0,56956,NaN


In [63]:
df2.loc[realtimeindex, 'settlement_time'] = df2['last_updated_datetime']

In [64]:
df2[df2['settlement_time'].isna()]

,created_datetime,last_updated_datetime,settlement_time,source_of_fund,service,vendor_code,transaction_id,tx_ref_number,tx_serial_number,amount,username,transaction_status,vendor_service,admin_fee,admin_fee_invoice,deduction_cost,settlement_amount,realtime?


In [65]:
df2.loc[realtimeindex, 'settlement_time'] = pd.to_datetime(df2['settlement_time'].dt.date)

In [66]:
df2.loc[realtimeindex,'realtime?'] = "Realtime"

In [67]:
nonrealtimeindex = df2[df2['realtime?'].isna()].index

In [68]:
df2.loc[nonrealtimeindex, 'realtime?'] = 'Non Realtime'

In [69]:
df2[df2['realtime?'].isna()]

,created_datetime,last_updated_datetime,settlement_time,source_of_fund,service,vendor_code,transaction_id,tx_ref_number,tx_serial_number,amount,username,transaction_status,vendor_service,admin_fee,admin_fee_invoice,deduction_cost,settlement_amount,realtime?


In [70]:
grouping2 = df2.groupby(['vendor_code','username', 'realtime?', 'last_updated_datetime', 'settlement_time']).agg({'settlement_amount' : 'sum'})

In [71]:
grouping2

settlement_amount
vendor_code username            realtime?    last_updated_datetime settlement_time                       
DANA        abhitech            Non Realtime 2024-05-26            2024-05-27 12:00:00         1826883278
            angkasa2023         Non Realtime 2024-05-26            2024-05-27 12:00:00         3032607219
            aniswinarno         Non Realtime 2024-05-26            2024-05-27 12:00:00             125340
            anjanajayateknologi Non Realtime 2024-05-26            2024-05-27 12:00:00         2432726711
                                                                   2024-05-28 15:00:00             118188
            bayuangan           Non Realtime 2024-05-26            2024-05-27 12:00:00             177003
            bicolink            Non Realtime 2024-05-26            2024-05-27 12:00:00              49165
            dori123             Non Realtime 2024-05-26            2024-05-27 12:00:00              70000
            erprofaznet         Non Realtime 2024-05-26            2024-05-27 12:00:00            6104252
            gamingku            Realtime     2024-05-26            2024-05-26 00:00:00         4574249913
            greentime           Non Realtime 2024-05-26            2024-05-27 12:00:00           48809401
            gst2023             Realtime     2024-05-26            2024-05-26 00:00:00          599418360
            henryhendrawan      Non Realtime 2024-05-26            2024-05-27 12:00:00             529392
            hezheng2023         Non Realtime 2024-05-26            2024-05-27 12:00:00           67829104
            hokkaidosweet       Non Realtime 2024-05-26            2024-05-27 12:00:00           17498459
            humphryho           Non Realtime 2024-05-26            2024-05-27 12:00:00           17065880
            nbatech             Realtime     2024-05-26            2024-05-26 00:00:00         1797323042
            nfjuara             Non Realtime 2024-05-26            2024-05-27 12:00:00              20010
            novanadrian         Non Realtime 2024-05-26            2024-05-27 12:00:00             155000
            profezorrr          Non Realtime 2024-05-26            2024-05-27 12:00:00             149500
            pulsapedia          Realtime     2024-05-26            2024-05-26 00:00:00           57542679
            skyzeevoucher       Non Realtime 2024-05-26            2024-05-27 12:00:00              49000
LINKAJA     angkasa2023         Non Realtime 2024-05-26            2024-05-27 12:00:00          229063057
            bicolink            Non Realtime 2024-05-26            2024-05-27 12:00:00               9833
            erprofaznet         Non Realtime 2024-05-26            2024-05-27 12:00:00             859042
            gst2023             Realtime     2024-05-26            2024-05-26 00:00:00           17357817
            humphryho           Non Realtime 2024-05-26            2024-05-27 12:00:00           10544159
            labamuadmin         Non Realtime 2024-05-26            2024-05-27 12:00:00             116526
            monetapay           Non Realtime 2024-05-26            2024-05-27 12:00:00            1671658
            nbatech             Realtime     2024-05-26            2024-05-26 00:00:00           95256615
            payermax            Non Realtime 2024-05-26            2024-05-27 12:00:00           63753602
            payhere             Realtime     2024-05-26            2024-05-26 00:00:00           51009056
            pulsapedia          Realtime     2024-05-26            2024-05-26 00:00:00             771466
OVO         abhitech            Non Realtime 2024-05-26            2024-05-28 15:00:00           48919745
            erprofaznet         Non Realtime 2024-05-26            2024-05-28 15:00:00             535638
            gst2023             Realtime     2024-05-26            2024-05-26 00:00:00          123335448
            henryhendrawan      Non Realtime 2024-05-26     

In [72]:
grouping2 = grouping2.reset_index()

In [73]:
grouping2.rename(columns={'vendor_code' : 'Vendor Name', 'last_updated_datetime' : 'Transaction Date'}, inplace=True)

In [74]:
grouping2 = grouping2.sort_values(by=['realtime?'])

In [75]:
linkaja = grouping2[grouping2['Vendor Name'] == "LINKAJA"]

In [76]:
danaewallet = grouping2[grouping2['Vendor Name'] == "DANA"]

In [77]:
ovo = grouping2[grouping2['Vendor Name'] == "OVO"]

In [78]:
linkaja['settlement_amount'].sum() + danaewallet['settlement_amount'].sum() + ovo['settlement_amount'].sum()

15915213738

Start VA Vendor

In [79]:
df3 = pd.read_csv('VA Vendor 26 May 24.csv')

In [80]:
df3

,created_datetime,last_updated_datetime,settlement_time,settlement_amount,amount,service,sender_bank,vendor,transaction_status,username,tx_id,ref_number,unique_id,va_number,admin_fee,admin_fee_invoice,deduction_cost
0,"May 26, 2024, 11:32:54 PM","May 26, 2024, 11:32:54 PM","May 27, 2024, 12:00:00 PM",995500,1000000,VA Aggregator,008,DOKU,SUCCESS,adiputra44,0f834691-6fa3-4032-91db-232200d52beb,2405P3TQVH4O,2405P3TQVH4O,8889990323272747,4500,0,1450
1,"May 26, 2024, 11:31:14 PM","May 26, 2024, 11:31:14 PM","May 27, 2024, 12:00:00 PM",95500,100000,VA Aggregator,008,DOKU,SUCCESS,tangankanan,932de4bc-9860-436c-8689-a08d4543604a,240575627C7V,240575627C7V,8889990323272746,4500,0,1450
2,"May 26, 2024, 11:29:56 PM","May 26, 2024, 11:29:56 PM","May 27, 2024, 12:00:00 PM",695500,700000,VA Aggregator,008,DOKU,SUCCESS,tangankanan,e7b2e85e-89ad-4437-a351-8c3bf50679d8,2405CQCTYHP6,2405CQCTYHP6,8889990323272745,4500,0,1450
3,"May 26, 2024, 11:28:57 PM","May 26, 2024, 11:28:57 PM",NaN,68200,70000,VA Aggregator,008,DOKU,SUCCESS,unipin,9a7adff1-af53-4f3e-b536-8de2282bf067,2405HVSZJP8B,2405HVSZJP8B,8889990323272743,1800,0,1450
4,"May 26, 2024, 11:27:17 PM","May 26, 2024, 11:27:17 PM",NaN,82560,84360,VA Aggregator,008,DOKU,SUCCESS,unipin,bc68b6a0-c262-4a4e-89cf-c93168d1e35a,2405Z13FLTT2,2405Z13FLTT2,8889990323272742,1800,0,1450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,"May 26, 2024, 12:24:45 AM","May 26, 2024, 12:24:45 AM","May 27, 2024, 12:00:00 PM",95500,100000,VA Aggregator,008,DOKU,SUCCESS,tangankanan,037b53d3-4113-4e33-8392-7b1959e7ef6f,240507UBWHU3,240507UBWHU3,8889990323272565,4500,0,1450
542,"May 26, 2024, 12:24:07 AM","May 26, 2024, 12:24:07 AM",NaN,98200,100000,VA Aggregator,008,DOKU,SUCCESS,unipin,34fd331c-93b1-4ca6-9323-85b21da00522,2405UN1NB45S,2405UN1NB45S,8889990323272564,1800,0,1450
543,"May 26, 2024, 12:20:26 AM","May 26, 2024, 12:20:26 AM","May 27, 2024, 12:00:00 PM",3995500,4000000,VA Aggregator,008,DOKU,SUCCESS,tangankanan,dabe2a00-d871-4c76-996b-c2eedc5ca717,2405D33U4MJY,2405D33U4MJY,8889990323272562,4500,0,1450
544,"May 26, 2024, 12:17:36 AM","May 26, 2024, 12:17:36 AM",NaN,98200,100000,VA Aggregator,008,DOKU,SUCCESS,unipin,f81a6cd3-a49a-479f-bcdf-d09291c3d0a9,2405DQCJ70LS,2405DQCJ70LS,8889990323272561,1800,0,1450


In [81]:
df3['settlement_amount'].sum()

2220969377

In [82]:
df3['last_updated_datetime'] = pd.to_datetime(df3['last_updated_datetime'])

/var/folders/sg/v7nz4brs3sq_pn793b_nw89c0000gp/T/ipykernel_21311/4082101892.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df3['last_updated_datetime'] = pd.to_datetime(df3['last_updated_datetime'])


In [83]:
df3['settlement_time'] = pd.to_datetime(df3['settlement_time'])

In [84]:
df3['last_updated_datetime'] = pd.to_datetime(df3['last_updated_datetime']).dt.date

In [85]:
df3['realtime?'] = np.nan

In [86]:
isnull3 = df3[(df3['settlement_amount'].isna() & df3['settlement_time'].isna()) | df3['settlement_amount'].isna()]

In [87]:
isnull3['username'].unique()

array([], dtype=object)

In [88]:
realtimeindex = df3[df3['settlement_time'].isna() & df3['settlement_amount'].notna()].index

In [89]:
df3.loc[realtimeindex, 'settlement_time'] = df3['last_updated_datetime']

In [90]:
df3.loc[realtimeindex, 'settlement_time'] = pd.to_datetime(df3['settlement_time'].dt.date)

In [91]:
df3.loc[realtimeindex,'realtime?'] = "Realtime"

In [92]:
nonrealtimeindex = df3[df3['realtime?'].isna()].index

In [93]:
df3.loc[nonrealtimeindex, 'realtime?'] = 'Non Realtime'

In [94]:
df3

,created_datetime,last_updated_datetime,settlement_time,settlement_amount,amount,service,sender_bank,vendor,transaction_status,username,tx_id,ref_number,unique_id,va_number,admin_fee,admin_fee_invoice,deduction_cost,realtime?
0,"May 26, 2024, 11:32:54 PM",2024-05-26,2024-05-27 12:00:00,995500,1000000,VA Aggregator,008,DOKU,SUCCESS,adiputra44,0f834691-6fa3-4032-91db-232200d52beb,2405P3TQVH4O,2405P3TQVH4O,8889990323272747,4500,0,1450,Non Realtime
1,"May 26, 2024, 11:31:14 PM",2024-05-26,2024-05-27 12:00:00,95500,100000,VA Aggregator,008,DOKU,SUCCESS,tangankanan,932de4bc-9860-436c-8689-a08d4543604a,240575627C7V,240575627C7V,8889990323272746,4500,0,1450,Non Realtime
2,"May 26, 2024, 11:29:56 PM",2024-05-26,2024-05-27 12:00:00,695500,700000,VA Aggregator,008,DOKU,SUCCESS,tangankanan,e7b2e85e-89ad-4437-a351-8c3bf50679d8,2405CQCTYHP6,2405CQCTYHP6,8889990323272745,4500,0,1450,Non Realtime
3,"May 26, 2024, 11:28:57 PM",2024-05-26,2024-05-26 00:00:00,68200,70000,VA Aggregator,008,DOKU,SUCCESS,unipin,9a7adff1-af53-4f3e-b536-8de2282bf067,2405HVSZJP8B,2405HVSZJP8B,8889990323272743,1800,0,1450,Realtime
4,"May 26, 2024, 11:27:17 PM",2024-05-26,2024-05-26 00:00:00,82560,84360,VA Aggregator,008,DOKU,SUCCESS,unipin,bc68b6a0-c262-4a4e-89cf-c93168d1e35a,2405Z13FLTT2,2405Z13FLTT2,8889990323272742,1800,0,1450,Realtime
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,"May 26, 2024, 12:24:45 AM",2024-05-26,2024-05-27 12:00:00,95500,100000,VA Aggregator,008,DOKU,SUCCESS,tangankanan,037b53d3-4113-4e33-8392-7b1959e7ef6f,240507UBWHU3,240507UBWHU3,8889990323272565,4500,0,1450,Non Realtime
542,"May 26, 2024, 12:24:07 AM",2024-05-26,2024-05-26 00:00:00,98200,100000,VA Aggregator,008,DOKU,SUCCESS,unipin,34fd331c-93b1-4ca6-9323-85b21da00522,2405UN1NB45S,2405UN1NB45S,8889990323272564,1800,0,1450,Realtime
543,"May 26, 2024, 12:20:26 AM",2024-05-26,2024-05-27 12:00:00,3995500,4000000,VA Aggregator,008,DOKU,SUCCESS,tangankanan,dabe2a00-d871-4c76-996b-c2eedc5ca717,2405D33U4MJY,2405D33U4MJY,8889990323272562,4500,0,1450,Non Realtime
544,"May 26, 2024, 12:17:36 AM",2024-05-26,2024-05-26 00:00:00,98200,100000,VA Aggregator,008,DOKU,SUCCESS,unipin,f81a6cd3-a49a-479f-bcdf-d09291c3d0a9,2405DQCJ70LS,2405DQCJ70LS,8889990323272561,1800,0,1450,Realtime


In [95]:
grouping3 = df3.groupby(['vendor','username', 'realtime?', 'last_updated_datetime', 'settlement_time']).agg({'settlement_amount' : 'sum'})

In [96]:
grouping3

settlement_amount
vendor username        realtime?    last_updated_datetime settlement_time                       
ALTO   erproproxinet   Non Realtime 2024-05-26            2024-05-28 15:00:00             249120
       inovasidaya     Non Realtime 2024-05-26            2024-05-28 15:00:00         2039984919
       pulsapedia      Realtime     2024-05-26            2024-05-26 00:00:00             535611
DOKU   adiputra44      Non Realtime 2024-05-26            2024-05-27 12:00:00          104961123
       danielsuwahjo   Non Realtime 2024-05-26            2024-05-27 12:00:00              12780
       erprofaznet     Non Realtime 2024-05-26            2024-05-27 12:00:00             186115
       hansolo         Non Realtime 2024-05-26            2024-05-27 12:00:00           26982000
       henryhendrawan  Realtime     2024-05-26            2024-05-26 00:00:00             100000
       moxafinancial20 Realtime     2024-05-26            2024-05-26 00:00:00            1083010
       tangankanan     Non Realtime 2024-05-26            2024-05-27 12:00:00           16107500
       unipin          Realtime     2024-05-26            2024-05-26 00:00:00            8938591
IFP    hypeplus        Realtime     2024-05-26            2024-05-26 00:00:00            8179485
       merchantops     Non Realtime 2024-05-26            2024-05-27 12:00:00            3491123
       ymaioy1234      Non Realtime 2024-05-26            2024-05-27 12:00:00            8800000
       yusantriananda  Realtime     2024-05-26            2024-05-26 00:00:00            1358000

In [97]:
grouping3 = grouping3.reset_index()

In [98]:
grouping3.rename(columns={'vendor' : 'Vendor Name', 'last_updated_datetime' : 'Transaction Date'}, inplace=True)

In [99]:
grouping3

,Vendor Name,username,realtime?,Transaction Date,settlement_time,settlement_amount
0,ALTO,erproproxinet,Non Realtime,2024-05-26,2024-05-28 15:00:00,249120
1,ALTO,inovasidaya,Non Realtime,2024-05-26,2024-05-28 15:00:00,2039984919
2,ALTO,pulsapedia,Realtime,2024-05-26,2024-05-26 00:00:00,535611
3,DOKU,adiputra44,Non Realtime,2024-05-26,2024-05-27 12:00:00,104961123
4,DOKU,danielsuwahjo,Non Realtime,2024-05-26,2024-05-27 12:00:00,12780
5,DOKU,erprofaznet,Non Realtime,2024-05-26,2024-05-27 12:00:00,186115
6,DOKU,hansolo,Non Realtime,2024-05-26,2024-05-27 12:00:00,26982000
7,DOKU,henryhendrawan,Realtime,2024-05-26,2024-05-26 00:00:00,100000
8,DOKU,moxafinancial20,Realtime,2024-05-26,2024-05-26 00:00:00,1083010
9,DOKU,tangankanan,Non Realtime,2024-05-26,2024-05-27 12:00:00,16107500


In [100]:
grouping3 = grouping3.sort_values(by=['realtime?'])

In [101]:
grouping3

,Vendor Name,username,realtime?,Transaction Date,settlement_time,settlement_amount
0,ALTO,erproproxinet,Non Realtime,2024-05-26,2024-05-28 15:00:00,249120
1,ALTO,inovasidaya,Non Realtime,2024-05-26,2024-05-28 15:00:00,2039984919
3,DOKU,adiputra44,Non Realtime,2024-05-26,2024-05-27 12:00:00,104961123
4,DOKU,danielsuwahjo,Non Realtime,2024-05-26,2024-05-27 12:00:00,12780
5,DOKU,erprofaznet,Non Realtime,2024-05-26,2024-05-27 12:00:00,186115
6,DOKU,hansolo,Non Realtime,2024-05-26,2024-05-27 12:00:00,26982000
9,DOKU,tangankanan,Non Realtime,2024-05-26,2024-05-27 12:00:00,16107500
12,IFP,merchantops,Non Realtime,2024-05-26,2024-05-27 12:00:00,3491123
13,IFP,ymaioy1234,Non Realtime,2024-05-26,2024-05-27 12:00:00,8800000
2,ALTO,pulsapedia,Realtime,2024-05-26,2024-05-26 00:00:00,535611


In [102]:
ifp = grouping3[grouping3['Vendor Name'] == "IFP"]

In [103]:
alto = grouping3[grouping3['Vendor Name'] == "ALTO"]

In [104]:
doku = grouping3[grouping3['Vendor Name'] == "DOKU"]

In [105]:
ifp['settlement_amount'].sum() + alto['settlement_amount'].sum() + doku['settlement_amount'].sum()

2220969377

Start Shopee

In [106]:
df4 = pd.read_csv('Shopee 26 May 24.csv')

In [107]:
df4

,Created Date,Last Updated Date,Transaction Amount,Username,settlement_time,settlement_amount,vendor,product_type,is_testing_account,Type of Transaction,Status,Referral ID,admin_fee,base_admin_fee,admin_fee_invoice,deduction_cost,deduction_cost_gross
0,"May 26, 2024, 11:59:45 PM","May 26, 2024, 11:59:58 PM",10000,humphryho,"May 27, 2024, 12:00:00 PM",9848,SHOPEEPAY,ewallet,False,SHOPEEPAY_JUMPAPP,SUCCESS,24056V8UQH05,152,137,0,90.000,90.000
1,"May 26, 2024, 11:55:45 PM","May 26, 2024, 11:56:05 PM",20000,humphryho,"May 27, 2024, 12:00:00 PM",19696,SHOPEEPAY,ewallet,False,SHOPEEPAY_JUMPAPP,SUCCESS,2405MM0QMZY8,304,274,0,180.000,180.000
2,"May 26, 2024, 11:49:32 PM","May 26, 2024, 11:49:54 PM",29000,instanshop,"May 27, 2024, 12:00:00 PM",28420,SHOPEEPAY,payment_routing,False,SHOPEEPAY_JUMPAPP,SUCCESS,24057YR8T58N,580,523,0,NaN,NaN
3,"May 26, 2024, 11:47:42 PM","May 26, 2024, 11:48:05 PM",50000,kaigames,"May 27, 2024, 12:00:00 PM",48862,SHOPEEPAY,ewallet,False,SHOPEEPAY_JUMPAPP,SUCCESS,24054HF81FAZ,1138,1025,0,450.000,450.000
4,"May 26, 2024, 11:47:40 PM","May 26, 2024, 11:47:57 PM",10000,humphryho,"May 27, 2024, 12:00:00 PM",9848,SHOPEEPAY,ewallet,False,SHOPEEPAY_JUMPAPP,SUCCESS,2405ST5TGYX0,152,137,0,90.000,90.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,"May 26, 2024, 12:04:36 AM","May 26, 2024, 12:04:49 AM",15829,humphryho,"May 27, 2024, 12:00:00 PM",15588,SHOPEEPAY,ewallet,False,SHOPEEPAY_JUMPAPP,SUCCESS,2405GX3SGSQN,241,217,0,142.461,142.461
512,"May 26, 2024, 12:04:32 AM","May 26, 2024, 12:05:28 AM",35000,bonophotobooth,"May 27, 2024, 12:00:00 PM",34728,SHOPEEPAY,payment_routing,False,QRIS,SUCCESS,0073841590501916,272,245,0,NaN,NaN
513,"May 26, 2024, 12:04:07 AM","May 26, 2024, 12:04:22 AM",400000,rupaktiger,"May 27, 2024, 12:00:00 PM",391120,SHOPEEPAY,ewallet,False,SHOPEEPAY_JUMPAPP,SUCCESS,24059A7DPXUW,8880,8000,0,3600.000,3600.000
514,"May 26, 2024, 12:02:37 AM","May 26, 2024, 12:02:50 AM",25000,rupaktiger,"May 27, 2024, 12:00:00 PM",24445,SHOPEEPAY,ewallet,False,SHOPEEPAY_JUMPAPP,SUCCESS,2405ABAYVY55,555,500,0,225.000,225.000


In [108]:
df4['settlement_amount'].sum()

32441795

In [109]:
df4['Last Updated Date'] = pd.to_datetime(df4['Last Updated Date'])

/var/folders/sg/v7nz4brs3sq_pn793b_nw89c0000gp/T/ipykernel_21311/796449051.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df4['Last Updated Date'] = pd.to_datetime(df4['Last Updated Date'])


In [110]:
df4['settlement_time'] = pd.to_datetime(df4['settlement_time'])

In [111]:
df4['Last Updated Date'] = pd.to_datetime(df4['Last Updated Date']).dt.date

In [112]:
df4['realtime?'] = np.nan

In [113]:
isnull4 = df4[(df4['settlement_amount'].isna() & df4['settlement_time'].isna()) | df4['settlement_amount'].isna()]

In [114]:
isnull4['Username'].unique()

array([], dtype=object)

In [115]:
realtimeindex = df4[df4['settlement_time'].isna() & df4['settlement_amount'].notna()].index

In [116]:
df4.loc[realtimeindex, 'settlement_time'] = df4['Last Updated Date']

In [117]:
df4.loc[realtimeindex, 'settlement_time'] = pd.to_datetime(df4['settlement_time'].dt.date)

In [118]:
df4.loc[realtimeindex,'realtime?'] = "Realtime"

In [119]:
nonrealtimeindex = df4[df4['realtime?'].isna()].index

In [120]:
df4.loc[nonrealtimeindex, 'realtime?'] = 'Non Realtime'

In [121]:
grouping4 = df4.groupby(['vendor','Username', 'realtime?', 'Last Updated Date', 'settlement_time']).agg({'settlement_amount' : 'sum'})

In [122]:
grouping4 = grouping4.reset_index()

In [123]:
grouping4.rename(columns={'vendor' : 'Vendor Name', 'Last Updated Date' : 'Transaction Date'}, inplace=True)

In [124]:
grouping4 = grouping4.sort_values(by=['realtime?'])

In [125]:
grouping4

,Vendor Name,Username,realtime?,Transaction Date,settlement_time,settlement_amount
0,SHOPEEPAY,asrul10,Non Realtime,2024-05-26,2024-05-27 12:00:00,79278
16,SHOPEEPAY,rupaktiger,Non Realtime,2024-05-26,2024-05-27 12:00:00,5937522
15,SHOPEEPAY,riswana,Non Realtime,2024-05-26,2024-05-27 12:00:00,9778
13,SHOPEEPAY,novanadrian,Non Realtime,2024-05-26,2024-05-27 12:00:00,185000
12,SHOPEEPAY,monetapay,Non Realtime,2024-05-26,2024-05-27 12:00:00,843837
11,SHOPEEPAY,matikct,Non Realtime,2024-05-26,2024-05-27 12:00:00,3254968
10,SHOPEEPAY,kaigames,Non Realtime,2024-05-26,2024-05-27 12:00:00,1155099
17,SHOPEEPAY,sewastudioid,Non Realtime,2024-05-26,2024-05-27 12:00:00,250000
9,SHOPEEPAY,instanshop,Non Realtime,2024-05-26,2024-05-27 12:00:00,43120
7,SHOPEEPAY,henryhendrawan,Non Realtime,2024-05-26,2024-05-27 12:00:00,1126349


In [126]:
shopee = grouping4[grouping4['Vendor Name'] == "SHOPEEPAY"]

In [127]:
shopee['settlement_amount'].sum()

32441795

In [128]:
with pd.ExcelWriter("Deposit in transit Trx Date 26 May 24.xlsx") as writer:
    nobu.to_excel(writer, sheet_name="Nobu", index=False)
    danamon.to_excel(writer, sheet_name="Danamon", index=False)
    bagi.to_excel(writer, sheet_name="BAGI", index=False)
    dana.to_excel(writer, sheet_name="Qris Dana", index=False)
    gv.to_excel(writer, sheet_name="GV", index=False)
    ovo.to_excel(writer, sheet_name="Ovo", index=False)
    danaewallet.to_excel(writer, sheet_name="Dana E-Wallet", index=False)
    linkaja.to_excel(writer, sheet_name="Linkaja", index=False)
    ifp.to_excel(writer, sheet_name="IFP", index=False)
    alto.to_excel(writer, sheet_name="Alto", index=False)
    doku.to_excel(writer, sheet_name="Doku", index=False)
    shopee.to_excel(writer, sheet_name="Shopee", index=False)

In [129]:
print("Done")

Done
